In [1]:
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation, leaguedashteamstats, leagueleaders, leaguestandingsv3, teamgamelog, boxscoresummaryv2
from nba_api.stats.static import teams, players
import pandas as pd
import numpy as np
import json, csv, requests

In [5]:
season = "2024-25"
teamGames = cumestatsteamgames.CumeStatsTeamGames(league_id='00', season=season, season_type_all_star="Regular Season", team_id=1610612757).get_normalized_json()
teamGames = pd.DataFrame(json.loads(teamGames)['CumeStatsTeamGames'])
teamGames['SEASON']=season
teamGames

,MATCHUP,GAME_ID,SEASON
0,04/13/2025 Lakers at Trail Blazers,0022401199,2024-25
1,04/11/2025 Warriors at Trail Blazers,0022401183,2024-25
2,04/09/2025 Trail Blazers at Jazz,0022401162,2024-25
3,04/06/2025 Spurs at Trail Blazers,0022401139,2024-25
4,04/04/2025 Trail Blazers at Bulls,0022401122,2024-25
...,...,...,...
77,10/30/2024 Trail Blazers at Clippers,0022400127,2024-25
78,10/28/2024 Trail Blazers at Kings,0022400112,2024-25
79,10/27/2024 Pelicans at Trail Blazers,0022400099,2024-25
80,10/25/2024 Pelicans at Trail Blazers,0022400086,2024-25


In [24]:
stats_base = leaguedashteamstats.LeagueDashTeamStats(season='2024-25', season_type_all_star='Regular Season', measure_type_detailed_defense='Advanced').get_data_frames()[0]
stats_base.to_csv("team_stats_2024.csv", index=False)

In [31]:
league_leaders = leagueleaders.LeagueLeaders(season="2024-25", season_type_all_star="Playoffs", stat_category_abbreviation="PTS")
leader_data = league_leaders.get_data_frames()[0].head()
leader_data

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,1628983,1,Shai Gilgeous-Alexander,1610612760,OKC,18,660,188,397,0.474,...,101,122,32,10,40,48,548,541,3.05,0.80
1,1628973,2,Jalen Brunson,1610612752,NYK,18,680,181,393,0.461,...,62,126,8,5,55,59,530,443,2.29,0.14
2,1626157,3,Karl-Anthony Towns,1610612752,NYK,18,639,139,285,0.488,...,209,24,13,12,40,78,386,443,0.60,0.33
3,1630162,4,Anthony Edwards,1610612750,MIN,15,585,135,298,0.453,...,117,82,17,11,39,32,380,380,2.10,0.44
4,1627783,5,Pascal Siakam,1610612754,IND,18,601,141,270,0.522,...,109,59,17,10,21,53,371,388,2.81,0.81


In [18]:
league_standings = leaguestandingsv3.LeagueStandingsV3(season="2024-25", season_type="Regular Season")
league_standings_df = league_standings.get_data_frames()[0]
league_standings_df[['HOME_WINS', 'HOME_LOSSES']] = league_standings_df['HOME'].str.split("-", expand=True).astype(int)
league_standings_df['HOME_WIN_PCT'] = round(league_standings_df['HOME_WINS'] / (league_standings_df['HOME_WINS'] + league_standings_df['HOME_LOSSES']),2)

league_standings_df[['ROAD_WINS', 'ROAD_LOSSES']] = league_standings_df['ROAD'].str.split("-", expand=True).astype(int)
league_standings_df['ROAD_WIN_PCT'] = round(league_standings_df['ROAD_WINS'] / (league_standings_df['ROAD_WINS'] + league_standings_df['ROAD_LOSSES']),2)
league_standings_df.to_csv("team_standings_2024_reg.csv", index=False)

In [21]:
team_game_log_ind_playoffs_2024 = teamgamelog.TeamGameLog(season='2024-25', season_type_all_star="Playoffs", team_id='1610612754')
team_game_log_df = team_game_log_ind_playoffs_2024.get_data_frames()[0][0:10]
team_game_log_df.describe(include=np.number)

,Team_ID,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
count,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,...,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,1.610613e+09,2.400000,0.900000,0.773400,242.500000,41.500000,84.600000,0.48810,13.500000,33.600000,...,0.78140,7.600000,31.700000,39.300000,26.600000,7.600000,5.300000,13.400000,24.700000,116.200000
std,0.000000e+00,1.173788,0.737865,0.177421,7.905694,6.637436,7.545418,0.04334,3.719319,4.575296,...,0.05178,2.590581,6.056218,7.211873,4.812022,2.951459,2.406011,5.081557,2.162817,14.077563
min,1.610613e+09,1.000000,0.000000,0.500000,240.000000,30.000000,74.000000,0.40500,5.000000,25.000000,...,0.71400,5.000000,25.000000,32.000000,20.000000,1.000000,2.000000,7.000000,22.000000,94.000000
25%,1.610613e+09,1.250000,0.250000,0.667000,240.000000,37.500000,82.000000,0.45725,13.000000,30.500000,...,0.73575,6.000000,28.000000,35.250000,24.500000,6.000000,3.250000,10.250000,23.000000,108.000000
50%,1.610613e+09,2.500000,1.000000,0.750000,240.000000,42.000000,82.500000,0.50500,14.500000,34.000000,...,0.78650,7.000000,29.500000,37.500000,26.000000,8.500000,5.000000,12.000000,24.500000,114.000000
75%,1.610613e+09,3.000000,1.000000,0.950000,240.000000,45.750000,87.250000,0.51625,15.000000,36.500000,...,0.82550,8.000000,32.000000,39.750000,28.500000,9.750000,7.000000,14.500000,27.000000,128.000000
max,1.610613e+09,4.000000,2.000000,1.000000,265.000000,51.000000,100.000000,0.54100,18.000000,40.000000,...,0.84400,13.000000,43.000000,56.000000,37.000000,11.000000,9.000000,24.000000,27.000000,138.000000


In [49]:
team_game_log_ind_reg_2024 = teamgamelog.TeamGameLog(season='2024-25', season_type_all_star="Regular Season", team_id='1610612754')
team_game_log_ind_reg_2024_df = team_game_log_ind_reg_2024.get_data_frames()[0][0:10]
team_game_log_ind_reg_2024_df

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612754,0022401189,"APR 13, 2025",IND @ CLE,W,50,32,0.610,290,48,...,0.909,14,47,61,27,15,10,13,20,126
1,1610612754,0022401172,"APR 11, 2025",IND vs. ORL,L,49,32,0.605,240,40,...,0.657,17,27,44,24,8,7,17,19,115
2,1610612754,0022401168,"APR 10, 2025",IND vs. CLE,W,49,31,0.613,240,43,...,0.700,10,36,46,28,3,11,9,16,114
3,1610612754,0022401148,"APR 08, 2025",IND vs. WAS,W,48,31,0.608,240,33,...,0.784,13,36,49,21,7,5,7,16,104
4,1610612754,0022401142,"APR 06, 2025",IND @ DEN,W,47,31,0.603,240,46,...,0.857,7,34,41,34,10,7,6,17,125
5,1610612754,0022401119,"APR 04, 2025",IND vs. UTA,W,46,31,0.597,240,51,...,0.684,11,40,51,38,8,17,7,17,140
6,1610612754,0022401104,"APR 02, 2025",IND vs. CHA,W,45,31,0.592,240,47,...,0.700,11,27,38,29,10,6,6,16,119
7,1610612754,0022401090,"MAR 31, 2025",IND vs. SAC,W,44,31,0.587,240,42,...,1.000,6,34,40,28,6,5,12,14,111
8,1610612754,0022401079,"MAR 29, 2025",IND @ OKC,L,43,31,0.581,240,39,...,0.731,8,35,43,27,6,6,11,16,111
9,1610612754,0022401061,"MAR 27, 2025",IND @ WAS,W,43,30,0.589,240,59,...,0.895,13,37,50,48,8,2,12,19,162


In [9]:
game_id = "0022401139"
summary = boxscoresummaryv2.BoxScoreSummaryV2(game_id=game_id)
inactive_df = summary.inactive_players.get_data_frame()
inactive_df

,PLAYER_ID,FIRST_NAME,LAST_NAME,JERSEY_NUM,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION
0,1628368,De'Aaron,Fox,4,1610612759,San Antonio,Spurs,SAS
1,1630561,David,Duke Jr.,7,1610612759,San Antonio,Spurs,SAS
2,1631110,Jeremy,Sochan,10,1610612759,San Antonio,Spurs,SAS
3,1631127,Harrison,Ingram,55,1610612759,San Antonio,Spurs,SAS
4,1641705,Victor,Wembanyama,1,1610612759,San Antonio,Spurs,SAS
5,1642434,Riley,Minix,27,1610612759,San Antonio,Spurs,SAS
6,1631121,Bryce,McGowens,11,1610612757,Portland,Trail Blazers,POR
7,1629028,Deandre,Ayton,2,1610612757,Portland,Trail Blazers,POR
8,1629057,Robert,Williams III,35,1610612757,Portland,Trail Blazers,POR
